In [126]:
import pyvisa as visa
import time
import numpy as np
import matplotlib.pyplot as plt

In [127]:
rm = visa.ResourceManager()

In [128]:
rm.list_resources()

('ASRL10::INSTR', 'GPIB0::7::INSTR', 'GPIB0::11::INSTR')

In [129]:
inst_top = rm.open_resource('GPIB0::7::INSTR')
inst_bot = rm.open_resource('GPIB0::11::INSTR')
my_inst = [inst_top,inst_bot]

In [130]:
for inst in my_inst:
    
    inst.write(':OUTP1 ON') # Output1 ON
    inst.write(':OUTP1:IMP 50OHM') # Output Impedance
    inst.write(':OUTP1:POL INV') # Invert Output (negative polarity)
    inst.write(':PULS:TRAN1 1.9NS')

In [131]:
inst_top.write(':ARM:SOUR EXT1') # External Trigger
inst_top.write(':ARM:LEV -0.5V') # Trigger Level
inst_top.write(':PULS:DEL1 0NS') # Pulse Delay
inst_top.write(':PULS:WIDT1 20NS') # Pulse Width

inst_top.write(':VOLT1:HIGH 0V') # Voltage High
inst_top.write(':VOLT1:LOW -0.4V') # Voltage Low

(18, <StatusCode.success: 0>)

In [132]:
inst_bot.write(':ARM:SOUR EXT1') # External Trigger
inst_bot.write(':ARM:LEV -0.5V') # Trigger Level
inst_bot.write(':PULS:DEL1 0NS') # Pulse Delay
inst_bot.write(':PULS:WIDT1 20NS') # Pulse Width

inst_bot.write(':VOLT1:LIM:STAT OFF') # High Voltage Limit
inst_bot.write(':VOLT1:HIGH 0V') # Voltage High
inst_bot.write(':VOLT1:LOW -0.4V') # Voltage Low


# top and bot has relative delay of ~ 30 ns, top one is earlier

(18, <StatusCode.success: 0>)

In [63]:
# Prepare list of delays to be scanned over, go and fro

offset = 0 # ns
step = 5 # ns
time_span = 320 # ns
num_point = int(time_span/step)
t = np.zeros((2,num_point*4+12))
t[0] = np.concatenate(([0,0,0,0,0,0],
                       np.linspace(0.5,time_span,num_point),
                       np.linspace(time_span-step,0,num_point),
                       np.linspace(0.5,time_span,num_point),
                       np.linspace(time_span-step,0,num_point),
                       [0,0,0,0,0,0]))
t[0] = t[0]+offset
print('Estimated time is %d sec, or %d min' 
      % (len(t[0])-1,(len(t[0])-1)/60))

Estimated time is 3051 sec, or 50 min


In [192]:
# Prepare list of delays to be scanned over, one time only

offset =0 # ns
step = 5 # ns
pause = 1 # s
time_span = 400 # ns
num_point = int(time_span/step)
t = np.zeros((3,num_point+12))
t[0] = np.concatenate(([0,0,0,0,0,0],
                       np.linspace(step,time_span,num_point),
                       [0,0,0,0,0,0]))
t[0] = t[0]+offset
print('Estimated time is %d sec, or %d min' 
      % ((len(t[0])-1)*pause,(len(t[0])-1)*pause/60))

Estimated time is 91 sec, or 1 min


In [274]:
a = t[0]

In [193]:
# Send the list of delays to pulse generator to start the scan

inst_bot.write(':OUTP1 ON')
start_time = time.time()
for ii in range(len(t[0])-1):
    
    t[2][ii] = time.time()-start_time
    inst_bot.write(':PULS:DEL1 '+str(t[0][ii]) +'NS')
    t[1][ii] = time.time()-start_time
    time.sleep(pause)

t[1][-1] = time.time()-start_time
inst_bot.write(':OUTP1 OFF')

print('Actual elapsed time is %d sec' %(t[1][-1]))

Actual elapsed time is 91 sec


In [70]:
inst_top.write(':OUTP1 OFF')


(12, <StatusCode.success: 0>)

In [188]:
inst_bot.write(':OUTP1 ON')
inst_bot.write(':PULS:DEL1 320NS')


(18, <StatusCode.success: 0>)

In [45]:
hbar = 1.05*10**(-34) # J.s
ev = 1.5*10**(-19) #J
ph_qd = 0.751*ev # 1.6 micron

In [51]:
p = 0.005*10**(-9) # W
pulse_freq = 2.5 # MHz
ph_rate = p/(ph_qd)/(10**6) # MHz
ph_per_pulse = ph_rate/pulse_freq # Hz
ph_per_pulse

17.75410563692854

In [319]:
4.88/(np.log(1082/905))

27.318807550276002